<a href="https://colab.research.google.com/github/visiont3lab/streamlit-example-machine-learning/blob/main/Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error  # MSE
from sklearn.pipeline import Pipeline
import plotly.graph_objects as go

In [13]:
# Create a class to select numerical or categorical columns 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error  # MSE
from sklearn.metrics import mean_absolute_error # MAE
from sklearn.metrics import median_absolute_error # MedAE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RANSACRegressor, SGDRegressor, HuberRegressor, TheilSenRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
import joblib
import seaborn as sn
import matplotlib.pyplot as plt
import random
import numpy as np
import plotly.graph_objects as go
import pickle
import json
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn import datasets
from google.colab import data_table
import seaborn as sns
import plotly.express as px

## Collect the data

In [2]:
# X1        X2                  X3         Y 
# age  num hours working     experience   salary

X1 = np.random.randint(18,65,200)
X2 = np.random.randint(0,12,200)
X3 = np.random.randint(0,5,200)
Y = np.random.randint(500,3000,200)
X = np.array([X1,X3,X3]).T

T = np.array([X1,X2,X2,Y]).T

df = pd.DataFrame(T, columns=["age","num_hours","experience","salary"])
df.to_csv("name.csv")

## Fast training

In [ ]:
# Model selection
models = {}
models["Linear"]        = LinearRegression()
models["Huber"]         = HuberRegressor(max_iter=1000)
#models["TheilSen"]      = TheilSenRegressor()
models["Ridge"]         = Ridge()
models["Lasso"]         = Lasso()
models["ElasticNet"]    = ElasticNet()
models["KNN"]           = KNeighborsRegressor(n_neighbors=5)
models["DecisionTree"]  = DecisionTreeRegressor()
models["SVR"]           = SVR(gamma="auto")
models["AdaBoost"]      = AdaBoostRegressor(n_estimators=50)
models["GradientBoost"] = GradientBoostingRegressor(n_estimators=100)
models["RandomForest"]  = RandomForestRegressor(n_estimators=100)
models["ExtraTrees"]    = ExtraTreesRegressor(n_estimators=100)


for name in list(models.keys()):
  pipe = Pipeline([
      ("sc", StandardScaler()),
      ("model", models[name]) 
  ])

  X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.20,shuffle=True, random_state=5)

  pipe.fit(X_train, Y_train)

  Y_hat = pipe.predict(X_test)

  err = np.sqrt( mean_squared_error(Y_hat,Y_test) )

  print(name + " RMSE: ", err)


In [29]:
pipe = Pipeline([
      ("sc", StandardScaler()),
      ("model", models["ElasticNet"]) 
  ])

X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.20,shuffle=True, random_state=5)

pipe.fit(X_train, Y_train)

Y_hat = pipe.predict(X_test)

err = np.sqrt( mean_squared_error(Y_hat,Y_test) )

print(name + " RMSE: ", err)

joblib.dump(pipe, "model_fast.pkl")

Y_hat = pipe.predict(X)

x = np.linspace(start=0,stop=len(Y_hat),num=len(Y_hat))
fig = go.Figure()
fig.add_trace( go.Scatter( x=x, y=Y.flatten(),mode="markers+lines", name="Y") )
fig.add_trace( go.Scatter( x=x, y=Y_hat.flatten(),mode="markers+lines", name="Y_hat") )
fig.update_layout(title_text="Results", hovermode="x")
fig.show()

ExtraTrees RMSE:  740.6617948032859


## Cross validation 

In [ ]:

# Model selection
models = {}
models["Linear"]        = LinearRegression()
models["Huber"]         = HuberRegressor(max_iter=1000)
#models["TheilSen"]      = TheilSenRegressor()
models["Ridge"]         = Ridge()
models["Lasso"]         = Lasso()
models["ElasticNet"]    = ElasticNet()
models["KNN"]           = KNeighborsRegressor(n_neighbors=5)
models["DecisionTree"]  = DecisionTreeRegressor()
models["SVR"]           = SVR(gamma="auto")
models["AdaBoost"]      = AdaBoostRegressor(n_estimators=50)
models["GradientBoost"] = GradientBoostingRegressor(n_estimators=100)
models["RandomForest"]  = RandomForestRegressor(n_estimators=100)
models["ExtraTrees"]    = ExtraTreesRegressor(n_estimators=100)

# Cors
k_fold  = KFold(n_splits=5, random_state=5,shuffle=True)

model_results = []
model_names   = []
for name in list(models.keys()):
  
  # Model selection
  model_names.append(name)
  # Definisco il modello
  pipe = Pipeline([
      ("sc", StandardScaler()), # fit # mean, std
      ("model", models[name]) # trovare i parametri a0 a1 a2 
  ])
  c = 0 

  # Cross validation
  results = []
  for train_index, test_index in k_fold.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train = X[train_index]
    y_train = Y[train_index]
    X_test = X[test_index]
    y_test = Y[test_index]

    pipe.fit(X_train,y_train)
    y_hat_test = pipe.predict(X_test)
    err = np.sqrt( mean_squared_error(y_hat_test, y_test)) # rmse
    #err = mean_absolute_error(y_hat_test, y_test)         # mae
    results.append(err)
    print(name + ": insieme: "+ str(c) + " RMSE: ", err)
    c = c+1

  model_results.append(results)

In [23]:
  fig = go.Figure()
  for name,res in zip(model_names,model_results):    
      fig.add_trace(go.Box(y=res,name=name, boxpoints='all'))
  fig.show()

In [30]:
## Rialleno --> Su tutto il dataset
pipe = Pipeline([
    ("sc", StandardScaler()), # fit # mean, std
    ("model", models["ElasticNet"]) # trovare i parametri a0 a1 a2 
])
# Su tutti i dati
pipe.fit(X,Y)
joblib.dump(pipe, "model_cross.pkl")

Y_hat = pipe.predict(X)

x = np.linspace(start=0,stop=len(Y_hat),num=len(Y_hat))
fig = go.Figure()
fig.add_trace( go.Scatter( x=x, y=Y.flatten(),mode="markers+lines", name="Y") )
fig.add_trace( go.Scatter( x=x, y=Y_hat.flatten(),mode="markers+lines", name="Y_hat") )
fig.update_layout(title_text="Results", hovermode="x")
fig.show()

## Deploy

In [31]:
m =joblib.load("model_cross.pkl")
#m =joblib.load("model_fast.pkl")
Y_hat = m.predict(X)
